In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import random

In [0]:

def load_DataModule():
  DataModule = drive.CreateFile({'id':'1OQiv4PxLTDKGUrsq15KF6VyHqyMq58mi'})
  DataModule.GetContentFile('DataUtilsModule.ipynb')
  %run DataUtilsModule.ipynb

load_DataModule()

# **TensorFlow Model**

# New Section

In [0]:
class TenserFlowModel(object):
    
    def load_data(self):
        raise NotImplementedError("Load_data() Need To be Implemented")
    
    def create_placeholders(self):
        raise NotImplementedError("create_placeholders() Needs to be Implemented")
        
    def initialize_parameters(self):
        raise NotImplementedError("initialize_parameters() Needs to be Implemented")
        
    def create_feed_dict(self , input_batch , label_batch):
        raise NotImplementedError("create_feed_dict() Needs to be Implemented")
        
    def add_forward_model(self , input_data):
        raise NotImplementedError("add_model() Needs to be Implemented")
        
    def add_cost_op(self , predict):
        raise NotImplementedError("add_loss_op() Needs to be Implemented")
    
    def run_epoch(self , sess , input_data , input_labels):
        raise NotImplementedError("run_epoch Needs to be Implemented")
    
    def fit(self):
        raise NotImplementedError("fit() Needs to be Implemented")
        
    def predict(self , sess , input_data , input_labels=None):
        raise NotImplementedError("predict() Needs to be Implemented")

In [0]:
class tensorFLowCnnModel(TenserFlowModel):
    def __init__(self):
        tf.set_random_seed(1)
        self.img_height = None
        self.img_width = None
        self.img_channel = None
        self.totalClasses = 5
        self.learning_parameter_shape = {}
        self.learning_parameters = {}
        self.non_learning_parameters = {}
        self.stride = None
        self.window_size = None
        self.gray_scale = False
        self.dropout = []
        self.fully_connented_dims = []
        self.learning_rate = 0.001
        self.num_epochs=10
        self.dataPath = '../data/working/'
        self.minibatch_size = 64
        self.m = None
        self.best_validation = 0.0
        self.checkpoint_save_path = '../data/CheckPoints/'
        self.last_improvement_iter = None
        self.improved_str = ''
#         self.last_saved_model = None
        tensorFLowCnnModel.graph_summary_number = 1
        #Funtion
        self.load_data()
        self.apply_one_hot()
        self.__set_shape_var_of_Eachlayers()
        
        
    def apply_one_hot(self):
        self.Y_train = tf.one_hot(self.Y_train , self.totalClasses)
        self.Y_test = tf.one_hot(self.Y_test , self.totalClasses)
        self.Y_dev = tf.one_hot(self.Y_dev , self.totalClasses)
        
        with tf.Session() as sess:
            self.Y_train , self.Y_test,self.Y_dev = sess.run([self.Y_train , self.Y_test , self.Y_dev]) 
#         print("After OneHot")
#         print(self.X_train.shape)
#         print(self.Y_train.shape)

#         print(self.X_test.shape)
#         print(self.Y_test.shape)

#         print(self.X_dev.shape)
#         print(self.Y_dev.shape)
            
    def load_data(self):
        #if Data is Already is stored
        if(utils.ifDataExits('X_Train' , self.dataPath)):
            self.X_train , self.X_test , self.X_dev , self.Y_train , self.Y_test , self.Y_dev \
                                                                        = utils.load_saved_data()
#             print(self.X_train.shape)
#             print(self.Y_train.shape)

#             print(self.X_test.shape)
#             print(self.Y_test.shape)

#             print(self.X_dev.shape)
#             print(self.Y_dev.shape)
        else:
            self.X_train,self.X_test,self.Y_train,self.Y_test,self.X_dev,self.Y_dev  \
                                    = utils.load_data(5 ,img_size=100 , batch_size=400) 
    def __set_shape_var_of_Eachlayers(self):
        #For Learning Parameters
        filter_shape = [
                            [4,4,1,8] ,  #Layer 1
                            [4,4,8,16] , #Layer 2
                            [4,4,16,32] ,#Layer 3 
                            [4,4,32,32] ,#Layer 4
                            [4,4,32,64]  #Layer 5
                        ]
        if not self.gray_scale:
            filter_shape[0][2] = 3 
        
        for index,shape in enumerate(filter_shape):
            layer = index+1
            self.learning_parameter_shape["W" + str(layer)] = shape
#         print("Shape :" , self.learning_parameter_shape)
        #For Non-Learning Parameters
        
        #[for first maxpool layer,for second,for third , ...]
        self.window_size = [2,2,2,2,2]
        #[stride of conv layer , stride for max pool layer]
        self.stride = [[1,2] , [1,2] , [1,2] , [1,2] , [1,2]]
        self.dropout = [1 , 1 ,0.7 , 1 , 1]
        
        #Shape For FullyConnected Layer
        self.fully_connented_dims = [128 , 64 , 5] #for last Layer except sofmax Layer
        
    def __create_placeholders(self):
        X = tf.placeholder(dtype=tf.float32 , 
                           shape=[None , self.img_height , self.img_width , self.img_channel] ,
                           name = 'input_values'
                          )
        Y = tf.placeholder(dtype=tf.float32 ,
                           shape=[None , self.totalClasses] ,
                           name="input_labels"
                          )
        return X,Y
    
    def __initialize_learning_parameters(self):
        layers = len(self.learning_parameter_shape)
        for l in range(1, layers+1):
            self.learning_parameters["W"+str(l)]= \
                                    tf.get_variable("W"+str(l) ,
                                                    self.learning_parameter_shape["W"+str(l)] ,
                                                    initializer=tf.contrib.layers.xavier_initializer(seed=1)
                                                   )
#         print("Parameters" , self.learning_parameters)
    def __initialize_non_learning_parameters(self):
        layers = len(self.stride)
        
        for l in range(1,layers + 1):
            self.non_learning_parameters["Conv_s"+str(l)] = self.stride[l-1][0]
            self.non_learning_parameters["Maxpool_s"+str(l)] = self.stride[l-1][1]
            self.non_learning_parameters["Window_size"+str(l)] = self.window_size[l-1]
            self.non_learning_parameters["dropout"+str(l)] = self.dropout[l-1]
            
        
    def __initialize_parameters(self):
        self.__set_shape_var_of_Eachlayers()
        
        self.__initialize_learning_parameters()
        self.__initialize_non_learning_parameters()

        
    def __create_feed_dict(self , input_batch , label_batch):
        feed_dict = {}
        feed_dict[self.X] = input_batch
        feed_dict[self.Y] = label_batch
        
        return feed_dict
    
    def layersBeforeFlatten(self, X):
        LP = self.learning_parameters
        NLP = self.non_learning_parameters
        layers = len(LP)
        
        for l in range(1,layers+1):
            #Extract All parameters Needed
            conv_s = NLP["Conv_s"+str(l)]
            f = NLP["Window_size"+str(l)]
            pool_s = NLP["Maxpool_s"+str(l)]
            curr_filter = LP["W"+str(l)]
#             print("curr_filter" , curr_filter)
                
            #CONV Layer
            s = conv_s
            Z = tf.nn.conv2d(X , 
                             curr_filter , 
                             strides=[1,s,s,1] , 
                             padding='SAME' , 
                             name="Conv_Layer_{}".format(l))
            
            #Activation
            A = tf.nn.relu(Z , 
                           name="Relu_{}".format(l))
            
            #Maxpool
            s = pool_s
            features = tf.nn.max_pool(A , 
                                      ksize=[1,f,f,1] , 
                                      strides=[1,s,s,1] ,
                                      padding='SAME' , 
                                      name="Max_pool_{}".format(l))
            features = tf.nn.dropout(features ,
                                     rate=1-self.dropout[l-1])
            X = features
        return features

    def LayersAfterFlatten(self , X):
        layers = len(self.fully_connented_dims)
        
        for l in range(1,layers+1):
            Z = tf.contrib.layers.fully_connected(X  , self.fully_connented_dims[l-1] , activation_fn=None)
        return Z
    
    def __add_forward_model(self , input_data):
        X = self.layersBeforeFlatten(input_data)
        
        X_flatten = tf.contrib.layers.flatten(X)
        
        Z = self.LayersAfterFlatten(X_flatten)
        
        return Z
    
    def add_cost_op(self , predict , Y):
        
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict , labels=Y))
        return cost
    
    def get_random_mini_batch(self ,mini_batch_size = 64):
        mini_batches = []
        
        #About Random actually data is already Shuffled

        num_compelete_minibatches = math.floor(self.m / mini_batch_size)
        for k in range(num_compelete_minibatches):
            mini_batch_X = self.X_train[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
            mini_batch_Y = self.Y_train[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
            mini_batch = (mini_batch_X, mini_batch_Y)
            mini_batches.append(mini_batch)
        
        if self.m%num_compelete_minibatches != 0:
            mini_batch_X = self.X_train[num_compelete_minibatches * mini_batch_size : self.m,:,:,:]
            mini_batch_Y = self.Y_train[num_compelete_minibatches * mini_batch_size : self.m,:]
            mini_batch = (mini_batch_X, mini_batch_Y)
            mini_batches.append(mini_batch)

        return mini_batches
    
    def run_epoch(self , sess , input_data , input_label):
        epoch_cost = 0
        minibatches = self.get_random_mini_batch()

        for minibatch in minibatches:
            mini_X , mini_Y = minibatch

            _ , temp_cost = sess.run([self.optimizer , self.cost] , 
                                     feed_dict = {self.X:mini_X , self.Y:mini_Y})
            epoch_cost += temp_cost
        
        return epoch_cost 
    
    def saveModel(self , sess , curr_validation_acc , epoch):
        saver = tf.train.Saver()
        #Saving The Model
        if(curr_validation_acc > self.best_validation):
            self.best_validation = curr_validation_acc

            self.last_improvement_iter = epoch
            
            save_path = "{}{}-{}-{}%/{}%".format(self.checkpoint_save_path ,
                                                  "my-model",
                                                  epoch ,
                                                  round(curr_validation_acc*100,2),
                                                  round(curr_validation_acc*100,3))
#             self.last_saved_model = save_path
            with open("{}last_model.txt".format(self.checkpoint_save_path) , "w") as handle:
                handle.write(save_path)
            
            saver.save(sess , save_path)
            
            self.improved_str = "*"
        else:
            self.improved_str = ''
        
    
    def fit(self):
        tf.reset_default_graph()
        
        self.m , self.img_height , self.img_width , self.img_channel = self.X_train.shape
        self.totalClasses = self.Y_train.shape[1]
        
        with tf.name_scope("initial_steps"):
            self.X,self.Y = self.__create_placeholders()
            self.__initialize_parameters()
        
        with tf.name_scope("forward_train"):
            Z = self.__add_forward_model(self.X)
        
        with tf.name_scope("cost_op"):
            self.cost = self.add_cost_op(Z ,self.Y)
        
        with tf.name_scope("train"):
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
        
        with tf.name_scope("accuracy"):
            correct_prediction = tf.equal(tf.argmax(Z,1) , tf.argmax(self.Y,1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction , "float"))
            
        with tf.name_scope("summarie_Training"):
            tf.summary.scalar("cost" , self.cost)
            tf.summary.scalar("acc" , accuracy)
            self.summary_train = tf.summary.merge_all()
            
        with tf.name_scope("summarie_Dev"):
            tf.summary.scalar("cost" , self.cost)
            tf.summary.scalar("acc" , accuracy)
            self.summary_dev = tf.summary.merge_all()
            
        with tf.name_scope("summarie_Test"):
            tf.summary.scalar("cost" , self.cost)
            tf.summary.scalar("acc" , accuracy)
            self.summary_test = tf.summary.merge_all()
            
        writer  = tf.summary.FileWriter("../tensorboard/./graphs/{}".format(tensorFLowCnnModel.graph_summary_number))
        tensorFLowCnnModel.graph_summary_number += 1
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            
            writer.add_graph(sess.graph)
            
            costs = []
            
            #Restore Model If Exixts
            try:
                print("{}last_model.txt".format(self.checkpoint_save_path))
                with open("{}last_model.txt".format(self.checkpoint_save_path) , "r") as file:
                    last_saved_model = file.read()
                    
                saver = tf.train.Saver()
                saver.restore(sess , last_saved_model)
            except:
                print("No Parameters Saved Yet")
            
            for epoch in range(self.num_epochs):
                epoch_cost  = self.run_epoch(sess , self.X_train , self.Y_train)

                curr_validation_acc = sess.run(accuracy , {self.X:self.X_dev , self.Y:self.Y_dev}) 
                
                
            
                runned_summary = sess.run(self.summary_train , {self.X:self.X_train , self.Y:self.Y_train})
                writer.add_summary(runned_summary , epoch)
                runned_summary = sess.run(self.summary_dev , {self.X:self.X_train , self.Y:self.Y_train})
                writer.add_summary(runned_summary , epoch)
                runned_summary = sess.run(self.summary_test , {self.X:self.X_train , self.Y:self.Y_train})
                writer.add_summary(runned_summary , epoch)
                
                self.saveModel(sess , curr_validation_acc , epoch)
                
                print("Cost after Epoch %i: %f" % (epoch + 1 , epoch_cost),
                      "=> Training Acc : %f" %(sess.run(accuracy , {self.X:self.X_train , self.Y:self.Y_train})),
                      "=> Dev Acc : %f" %(curr_validation_acc),
                      "=> Test Acc : %f" %(sess.run(accuracy , {self.X:self.X_test , self.Y:self.Y_test})),
                      self.improved_str
                     )
                
                
                costs.append(epoch_cost)
#                 writer.add_summary(self.summary)
        writer.close()
        plt.plot(range(1,self.num_epochs+1) , costs)
        plt.ylabel('cost')
        plt.xlabel('iteration')
        plt.show()
        

# **Keras Model**

In [0]:
class kerasModel():

  def create_model():
    raise NotImplementedError("create_model is not Implemented Yet!")
    
  def compile_model():
    raise NotImplementedError("compile_model is not Implemeted Yet!")
    
  def run_model():
    raise NotImplementedError("run_model is not Implemented Yet!")
    

In [0]:
class KerasCnn(kerasModel):
  def __createConvLayers():
    pass
 
  def __createDenseLayers():
    pass
  
  def create_callbacks():
    pass

In [0]:
import tensorflow as tf

#Callback Class 1
class onBeginCallback(tf.keras.callbacks.Callback):
  #logs contains batch_num and batch_size
  def __init__(self , on_train = False , on_test = False , on_epoch=False):
    self.on_trainBegin = on_train
    self.on_testBegin = on_test
    self.on_epochBegin = on_epoch
    
  def on_train_batch_begin(self,batch , logs=None):
    if(self.on_trainBegin):
      print("\non_train_batch_begin->Logs: {}\n".format(logs))
    
  def on_test_batch_begin(self,batch , logs=None):
    if(self.on_testBegin):
      print("\non_test_batch_begin->Logs: {}\n".format(logs))
  
  def on_epoch_begin(self, epoch, logs=None):
    if(self.on_epochBegin):
      print("\non_epoch_begin->Logs: {}\n".format(logs))
  

#Callback Class 2
class onEndCallback(tf.keras.callbacks.Callback):
  #logs contains batch_num , batch_size , loss and Matric
  def __init__(self , on_train = False , on_test = False , on_epoch=False):
    self.on_trainEnd = on_train
    self.on_testEnd = on_test
    self.on_epochEnd = on_epoch
  def on_train_batch_end(self,batch , logs=None):
    if(self.on_trainEnd):
      print("\non_train_batch_begin->Logs: {}\n".format(logs))
    
  def on_test_batch_end(self,batch , logs=None):
    if(self.on_testEnd):
      print("\non_test_batch_begin->Logs: {}\n".format(logs))
    
  def on_epoch_end(self, epoch, logs=None):
    if(self.on_epochEnd):
      print("\non_epoch_begin->Logs: {}\n".format(logs))
      

In [0]:
class kerasCnnModel(KerasCnn):
  def __init__(self, parametersDict , callbackDict):
    self.callbackDict = callbackDict
    self.parametersDict = parametersDict
    self.filters = parametersDict['filters']
    self.Conv_layers = parametersDict['layers'][0]
    self.Dense_layers = parametersDict['layers'][1]
    self.kernel_size = parametersDict['kernel_size']
    self.padding = parametersDict['padding']
    self.dense_layers_size = parametersDict['dense_layers_size']
    self.activation = parametersDict['activation']
    self.pool_size = parametersDict['pool_size']
    self.dropout = parametersDict['dropout']
#     self.Convdrop = parametersDict['Convdrop']
#     self.flattenDrop = parametersDict['flattenDrop']
    random.seed(1)
    
    ###RUN INITIAL FUNTIONS
    self.__create_callbacks()
    
  def __createConvLayers(self):
    for i in range(self.Conv_layers):
      if i != 0:
        self.model.add(Conv2D(filters = self.filters[i], kernel_size = (self.kernel_size[i],self.kernel_size[i]) , padding = self.padding[i][0]))
#         print("Conv2D(filters = {}, kernel_size = ({},{}) , padding = {})".format(self.filters[i] ,self.kernel_size[i] , self.kernel_size[i] , self.padding[i][0]))
      else:
        self.model.add(Conv2D(filters = self.filters[i], kernel_size = (self.kernel_size[i],self.kernel_size[i]) , input_shape = self.input_shape))
#         print("Conv2D(filters = {}, kernel_size = ({},{}) , padding = {})".format(self.filters[i] ,self.kernel_size[i] , self.kernel_size[i] , self.padding[i][0]))

      self.model.add(Activation(self.activation[i]))
#       print("Activation({})".format(self.activation[i]))
      self.model.add(MaxPooling2D(pool_size = (self.pool_size[i] , self.pool_size[i]) , padding = self.padding[i][1]))
#       self.model.add(Dropout(self.Convdrop[i]))
  
#       print("MaxPooling2D(pool_size = ({} , {}) , padding = {})".format(self.pool_size[i] ,self.pool_size[i] ,self.padding[i][1] ) )
#       print("")
      
      
  def __createDenseLayers(self):
    for i in range(self.Dense_layers):
      self.model.add(Dense(self.dense_layers_size[i]))
#       print("(Dense({})".format(self.dense_layers_size[i]))
      self.model.add(Activation(self.activation[self.Conv_layers + i]))
#       print("Activation({})".format((self.activation[self.Conv_layers + i] ) ) ) 
      if self.dropout!=False and i != (self.Dense_layers - 1):
        self.model.add(Dropout(self.dropout[i]))
#         print("Dropout({})".format(self.dropout[i]))
      
      
  
  def create_model(self , input_shape=None , saved_model = None):
    self.input_shape = input_shape
    if saved_model == None:      
      self.model = Sequential()
      self.__createConvLayers()
      self.model.add(Flatten())
#       if self.flattenDrop:
#         self.model.add(Dropout(self.flattenDrop))
#         print("Adding")
#       print("Not Adding")
      self.__createDenseLayers()
      return self.model
    else:
      self.model = saved_model
    
    
  
  
  def compile_model(self , loss="sparse_categorical_crossentropy"  , optimizer=tf.train.AdamOptimizer(0.001) , metrics=['accuracy']):
    self.model.compile(loss = loss,
                       optimizer = optimizer,
                       metrics = metrics
                      )
  @classmethod
  def getTotalSubfile(self,path):
    totalFiles = 0
    for dir_name in os.listdir(path):
      totalFiles += len(os.listdir(os.path.join(path , dir_name)))
    return totalFiles
  def run_model(self , path_to_dir ,  epochs=5 , batch_size=64 ):
    
    training_datagen = ImageDataGenerator(rescale= 1./255 , featurewise_center = False , featurewise_std_normalization=False)
    test_datagen = ImageDataGenerator(rescale= 1.255 , featurewise_center = False , featurewise_std_normalization=False)
    validation_datagen = ImageDataGenerator(rescale= 1./255 , featurewise_center = False , featurewise_std_normalization=False)
    
    training_dir = os.path.join(path_to_dir , 'Training')
    Test_dir = os.path.join(path_to_dir , 'Test')
    validation_dir = os.path.join(path_to_dir , 'Dev')
    
    
    training_generator = training_datagen.flow_from_directory(training_dir
                                             ,target_size=(256, 256)
                                             ,color_mode='rgb'
                                             ,class_mode='sparse'
                                             ,batch_size=batch_size
                                             ,shuffle=True
                                             ,seed = 1
                                            )
    
    validation_datagen = validation_datagen.flow_from_directory(
                                                        directory=validation_dir,
                                                        target_size=(256, 256),
                                                        color_mode="rgb",
                                                        batch_size=batch_size,
                                                        class_mode="sparse",
                                                        shuffle=True,
                                                        seed=1
                                                    )
    
    test_generator = test_datagen.flow_from_directory(
                                                      directory=Test_dir,
                                                      target_size=(256, 256),
                                                      color_mode="rgb",
                                                      batch_size=batch_size,
                                                      class_mode="sparse",
                                                      shuffle=True,
                                                      seed=1
                                                      )
    
    steps_per_epoch = math.ceil(self.getTotalSubfile(os.path.join(path_to_dir , "Training"))  / batch_size)
    step_size_validation = math.ceil(self.getTotalSubfile(os.path.join(path_to_dir , "Dev"))  / batch_size)
    print(steps_per_epoch)
    print(step_size_validation)
    
    
    history_callback = self.model.fit_generator(generator=training_generator
                                               ,steps_per_epoch=steps_per_epoch
                                               ,validation_data=validation_datagen
                                               ,validation_steps=step_size_validation
                                               ,epochs=epochs
                                               ,verbose=1
                                               ,callbacks=self.callbacks
                                               )
    
#     history_callback = self.model.fit_generator(image_gen.flow(X , Y , batch_size=batch_size) , steps_per_epoch=len(X)/batch_size , epoch = epochs)
#     for X_batch , Y_batch in image_gen.flow(X , Y , batch_size=batch_size):
#     history_callback = self.model.fit( 
#                                        X 
#                                       ,Y
#                                       ,batch_size=batch_size
#                                       ,epochs=epochs
#                                       ,validation_data = (X_dev , Y_dev)
#                                       ,callbacks=self.callbacks
#                                     )
  
  
  def evaluate_model(self , x_test , y_test):
    self.model.evaluate(x_test , y_test)
    
    
  def model_summary(self):
    print(self.model.summary())
    
  def load_model(self,path):
    return load_model(path)
    
  def __create_tensorboard_callback(self):
    return TensorBoard(log_dir=self.log_dir)
  
  
  def __create_ModelCheckpoint(self):
    i = 1
    while(True):
      modified_path = os.path.join(self.callbackDict['modelcheckpoint_path'] , 'model' + str(i) ) 
      if os.path.exists(modified_path):
        i += 1
        continue
      else:
        os.mkdir(modified_path)
        self.callbackDict['modelcheckpoint_path']  = modified_path
        break
    path = os.path.join(self.callbackDict['modelcheckpoint_path'] , 'model.{epoch:02d}-{val_acc:.2f}.hdf5')
    return ModelCheckpoint(filepath=path
                          ,monitor= 'val_acc'
                          ,verbose= 1
                          ,save_best_only=True
                          ,mode='max'
                          ,save_weights_only=False,
                           save_freq='epoch'
                          )
  
  
  def __create_callbacks(self , log_dir="/content/gdrive/My Drive/SeedApp/data/logs"):
    self.log_dir = log_dir
    self.callbacks = []
    
    if(self.callbackDict['custom']):
      self.callbacks.append(onBeginCallback())
      self.callbacks.append(onEndCallback())
      
    if(self.callbackDict['tensorboard']):
      cb = self.__create_tensorboard_callback()
      self.callbacks.append(cb)
      
    if(self.callbackDict['modelcheckpoint']):
      cb = self.__create_ModelCheckpoint()
      self.callbacks.append(cb)
    
    

# New Section